# Greenplum Database  Concepts Explained - Part 3

This is Part 3 of Greenplum Database  Concepts Explained, ***MPP Fundamentals and Partitioning***. 

- If you missed Part 1 (*Setup, Describe Input Dataset & Data Loading*) or wish to repeat, then click [here](AWS-GP-demo-1.ipynb).
- If you missed Part 2 (*Basic Table Functions*) or wish to repeat, then click [here](AWS-GP-demo-2.ipynb).

In [1]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

'Connected: gpadmin@gpadmin'

In [2]:
%%sql $DB_USER@$DB_SERVER
SHOW gp_autostats_mode;
ALTER DATABASE gpadmin SET gp_autostats_mode TO 'NONE';
SHOW gp_autostats_mode;

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
ON_NO_STATS


## 5. MPP Fundamentals

This topic provides an overview of how a MPP (*Massively Parallel Processing*) database such as Greenplum, processes queries. Understanding this process can be useful when writing and tuning queries.

Users issue queries to Greenplum Database as they would to any database management system. They connect to the database instance on the Greenplum master host using a client application such as `psql` and submit SQL statements.

### 5.1. Understanding Parallel Query Execution & Greenplum Query Optimizer

Greenplum creates a number of database processes to handle the work of a query. On the master, the query worker process is called the **query dispatcher** (QD). The QD is responsible for creating and dispatching the query plan. It also accumulates and presents the final results. On the segments, a query worker process is called a **query executor** (QE). A QE is responsible for completing its portion of work and communicating its intermediate results to the other worker processes.

There is at least one worker process assigned to each slice of the query plan. A worker process works on its assigned portion of the query plan independently. During query execution, each segment will have a number of processes working on the query in parallel.

The Greenplum Database cost-based optimizer evaluates many strategies for executing a query and chooses the least costly method. The optimizer produces plans based on statistics generated for tables. It is important to have accurate statistics to produce the best plan. The Greenplum optimizer takes into account factors such as,
- the number of rows in tables to be joined, 
- availability of indexes, and 
- cardinality of column data when calculating the costs of alternative execution plans. 

The optimizer also accounts for the location of the data, **preferring to perform as much of the work as possible on the segments and to minimize the amount of data that must be transmitted between segments to complete the query**.

Check the following example:

### Example 1: Forcing data into a single segment

In [3]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/5-1-amzn-reviews-by-marketplace.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_by_marketplace ; 
 
 CREATE TABLE demo . amzn_reviews_by_marketplace ( 
 marketplace text NULL , 
 customer_id int8 NULL , 
 review_id text NULL , 
 product_id text NULL , 
 product_parent int8 NULL , 
 product_title text NULL , 
 product_category text NULL , 
 star_rating int4 NULL , 
 helpful_votes int4 NULL , 
 total_votes int4 NULL , 
 vine text NULL , 
 verified_purchase text NULL , 
 review_headline text NULL , 
 review_body text NULL , 
 review_date date NULL 
 ) 
 DISTRIBUTED BY ( marketplace ); 
 
 INSERT INTO demo . amzn_reviews_by_marketplace 
 SELECT * FROM demo . amzn_reviews ;

In [4]:
query = !cat script/5-1-amzn-reviews-by-marketplace.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
103145273 rows affected.


[]

#### Let's compare the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*) data distribution across segments:

In [5]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-1-data-distribution-two-tables.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/5-1-data-distribution-two-tables.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT * 
 FROM ( SELECT gp_segment_id :: INT AS "Segment ID" , 
 count ( * ) :: INT AS "Row Count" , 
 'demo.amzn_reviews' AS "Table Name" 
 FROM 
 demo . amzn_reviews 
 GROUP BY 1 ) A 
 UNION ALL 
 SELECT * 
 FROM ( 
 SELECT gp_segment_id :: INT AS "Segment ID" , 
 count ( * ) :: INT AS "Row Count" , 
 'demo.amzn_reviews_by_marketplace' AS "Table Name" 
 FROM 
 demo . amzn_reviews_by_marketplace 
 GROUP BY 1 ) B 
 ORDER BY 1 , 3

25 rows affected.


Segment ID,Row Count,Table Name
0,4298868,demo.amzn_reviews
1,4296740,demo.amzn_reviews
2,4291976,demo.amzn_reviews
3,4297200,demo.amzn_reviews
4,4300819,demo.amzn_reviews
5,4297083,demo.amzn_reviews
6,4301096,demo.amzn_reviews
7,4298972,demo.amzn_reviews
8,4297364,demo.amzn_reviews
9,4296533,demo.amzn_reviews


### Let's compare the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [6]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-1.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ) 
 FROM demo . amzn_reviews ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..431.00 rows = 1 width = 8 ) 
 Rows out : 1 rows with 1955 ms to end . 
 -> Gather Motion 24 : 1 ( slice1 ; segments : 24 ) ( cost = 0.00..431.00 rows = 1 width = 8 ) 
 Rows out : 24 rows at destination with 1187 ms to first row , 1955 ms to end . 
 -> Aggregate ( cost = 0.00..431.00 rows = 1 width = 8 ) 
 Rows out : Avg 1.0 rows x 24 workers . Max 1 rows ( seg0 ) with 1194 ms to end . 
 -> Table Scan on amzn_reviews ( cost = 0.00..431.00 rows = 1 width = 1 ) 
 Rows out : Avg 4297719.7 rows x 24 workers . Max 4301096 rows ( seg6 ) with 0.090 ms to first row , 913 ms to end . 
 Slice statistics : 
 ( slice0 ) Executor memory : 322 K bytes . 
 ( slice1 ) Executor memory : 155 K bytes avg x 24 workers , 155 K bytes max ( seg0 ) . 
 Statement statistics : 
 Memory used : 446464 K bytes 
 Settings : optimizer_enable_associativity = on 
 Optimizer status : PQO version 3.52.0 
 Total runtime : 1955.339 ms 
 ( 16 rows )

In [7]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-2.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ) 
 FROM demo . amzn_reviews_by_marketplace ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 --------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..1543.82 rows = 1 width = 8 ) 
 Rows out : 1 rows with 26968 ms to end . 
 -> Gather Motion 24 : 1 ( slice1 ; segments : 24 ) ( cost = 0.00..1543.82 rows = 1 width = 8 ) 
 Rows out : 24 rows at destination with 0.467 ms to first row , 26968 ms to end . 
 -> Aggregate ( cost = 0.00..1543.82 rows = 1 width = 8 ) 
 Rows out : Avg 1.0 rows x 24 workers . Max 1 rows ( seg0 ) with 0.028 ms to first row , 0.029 ms to end . 
 -> Table Scan on amzn_reviews_by_marketplace ( cost = 0.00..1536.39 rows = 3995619 width = 1 ) 
 Rows out : 103145273 rows ( seg13 ) with 0.034 ms to first row , 20563 ms to end . 
 Slice statistics : 
 ( slice0 ) Executor memory : 322 K bytes . 
 ( slice1 ) Executor memory : 135 K bytes avg x 24 workers , 135 K bytes max ( seg0 ) . 
 Statement statistics : 
 Memory used : 446464 K bytes 
 Settings : optimizer_enable_associativity = on 
 Optimizer status : PQO version 3.52.0 
 Total runtime : 26969.025 ms 
 ( 16 rows )

### Let's run `ANALYZE <tablename>` to collect table statistics, and compare again the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [8]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-3.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-3.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYSE demo . amzn_reviews ; 
 
 EXPLAIN ANALYZE 
 SELECT COUNT(*) 
 FROM demo.amzn_reviews;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYZE 
 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..1636.42 rows = 1 width = 8 ) 
 Rows out : 1 rows with 1841 ms to end . 
 -> Gather Motion 24 : 1 ( slice1 ; segments : 24 ) ( cost = 0.00..1636.42 rows = 1 width = 8 ) 
 Rows out : 24 rows at destination with 1162 ms to first row , 1841 ms to end . 
 -> Aggregate ( cost = 0.00..1636.42 rows = 1 width = 8 ) 
 Rows out : Avg 1.0 rows x 24 workers . Max 1 rows ( seg0 ) with 1184 ms to end . 
 -> Table Scan on amzn_reviews ( cost = 0.00..1628.26 rows = 4388766 width = 1 ) 
 Rows out : Avg 4297719.7 rows x 24 workers . Max 4301096 rows ( seg6 ) with 0.081 ms to first row , 888 ms to end . 
 Slice statistics : 
 ( slice0 ) Executor memory : 322 K bytes . 
 ( slice1 ) Executor memory : 135 K bytes avg x 24 workers , 135 K bytes max ( seg0 ) . 
 Statement statistics : 
 Memory used : 446464 K bytes 
 Settings : optimizer_enable_associativity = on 
 Optimizer status : PQO version 3.52.0 
 Total runtime : 1841.317 ms 
 ( 16 rows )

In [9]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-4.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-4.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYSE demo . amzn_reviews_by_marketplace ; 
 
 EXPLAIN ANALYZE 
 SELECT COUNT(*) 
 FROM demo.amzn_reviews_by_marketplace;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYZE 
 QUERY PLAN 
 -------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..1443.83 rows = 1 width = 8 ) 
 Rows out : 1 rows with 26996 ms to end . 
 -> Gather Motion 24 : 1 ( slice1 ; segments : 24 ) ( cost = 0.00..1443.83 rows = 1 width = 8 ) 
 Rows out : 24 rows at destination with 0.482 ms to first row , 26996 ms to end . 
 -> Aggregate ( cost = 0.00..1443.83 rows = 1 width = 8 ) 
 Rows out : Avg 1.0 rows x 24 workers . Max 1 rows ( seg0 ) with 0.056 ms to end . 
 -> Table Scan on amzn_reviews_by_marketplace ( cost = 0.00..1437.08 rows = 3629435 width = 1 ) 
 Rows out : 103145273 rows ( seg13 ) with 0.030 ms to first row , 20578 ms to end . 
 Slice statistics : 
 ( slice0 ) Executor memory : 322 K bytes . 
 ( slice1 ) Executor memory : 135 K bytes avg x 24 workers , 135 K bytes max ( seg0 ) . 
 Statement statistics : 
 Memory used : 446464 K bytes 
 Settings : optimizer_enable_associativity = on 
 Optimizer status : PQO version 3.52.0 
 Total runtime : 26996.592 ms 
 ( 16 rows )

## 6. Partitioning

Table partitioning enables supporting very large tables, such as fact tables, by logically dividing them into smaller, more manageable pieces. Partitioned tables can improve query performance by allowing the Greenplum Database query optimizer to scan only the data needed to satisfy a given query instead of scanning all the contents of a large table.

### 6.1. Create a new copy of the original table, define a *PARTITION* pattern (by month) and load it.

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Rows that cannot be mapped to a child table partition are rejected and the load fails. To avoid unmapped rows being rejected at load time, define your partition hierarchy with a DEFAULT partition. Any rows that do not match a partition's CHECK constraints load into the DEFAULT partition.

At runtime, the query optimizer scans the entire table inheritance hierarchy and uses the CHECK table constraints to determine which of the child table partitions to scan to satisfy the query's conditions. The DEFAULT partition (if your hierarchy has one) is always scanned. DEFAULT partitions that contain data slow down the overall scan time.

When you use COPY or INSERT to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [10]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-1-create-and-load-partition-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-1-create-and-load-partition-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_partitioned ; 

 CREATE TABLE demo . amzn_reviews_partitioned ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id ) 
 PARTITION BY RANGE ( review_date )( 
 START ( '1995-01-01' :: date ) 
 END ( '2015-12-31' :: date ) INCLUSIVE 
 EVERY ( '1 year' :: interval ) 
 ); 

 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1995-01-01' ) TO year1995 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1996-01-01' ) TO year1996 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1997-01-01' ) TO year1997 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1998-01-01' ) TO year1998 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1999-01-01' ) TO year1999 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2000-01-01' ) TO year2000 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2001-01-01' ) TO year2001 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2002-01-01' ) TO year2002 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2003-01-01' ) TO year2003 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2004-01-01' ) TO year2004 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2005-01-01' ) TO year2005 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2006-01-01' ) TO year2006 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2007-01-01' ) TO year2007 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2008-01-01' ) TO year2008 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2009-01-01' ) TO year2009 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2010-01-01' ) TO year2010 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2011-01-01' ) TO year2011 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2012-01-01' ) TO year2012 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2013-01-01' ) TO year2013 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2014-01-01' ) TO year2014 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2015-01-01' ) TO year2015 ; 
 
 INSERT INTO demo . amzn_reviews_partitioned 
 SELECT * FROM demo . amzn_reviews ; 

 ANALYSE demo . amzn_reviews_partitioned ; 
 
 SELECT COUNT ( * ) AS row_count , 'demo.amzn_reviews' tablename FROM demo . amzn_reviews 
 UNION ALL 
 SELECT COUNT ( * ) AS row_count , 'demo.amzn_reviews_partitioned' tablename FROM demo . amzn_reviews_partitioned ;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
103145273 rows affected.
Done.
2 rows affected.


row_count,tablename
103145273,demo.amzn_reviews
103145273,demo.amzn_reviews_partitioned


### 6.2. Familiarize yourself with the Partitioned Table Design and Present Basic Demographics

#### 6.2.1. Retrieve Partitioned Table Design

In [11]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-1-partition-table-design.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-1-partition-table-design.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 partitionboundary , 
 partitiontablename , 
 partitionname , 
 partitionlevel , 
 partitionrank 
 FROM pg_partitions 
 WHERE tablename = 'amzn_reviews_partitioned' ;

21 rows affected.


partitionboundary,partitiontablename,partitionname,partitionlevel,partitionrank
PARTITION year1995 START ('1995-01-01'::date) END ('1996-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1995,year1995,0,1
PARTITION year1996 START ('1996-01-01'::date) END ('1997-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1996,year1996,0,2
PARTITION year1997 START ('1997-01-01'::date) END ('1998-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1997,year1997,0,3
PARTITION year1998 START ('1998-01-01'::date) END ('1999-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1998,year1998,0,4
PARTITION year1999 START ('1999-01-01'::date) END ('2000-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1999,year1999,0,5
PARTITION year2000 START ('2000-01-01'::date) END ('2001-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2000,year2000,0,6
PARTITION year2001 START ('2001-01-01'::date) END ('2002-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2001,year2001,0,7
PARTITION year2002 START ('2002-01-01'::date) END ('2003-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2002,year2002,0,8
PARTITION year2003 START ('2003-01-01'::date) END ('2004-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2003,year2003,0,9
PARTITION year2004 START ('2004-01-01'::date) END ('2005-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2004,year2004,0,10


#### 6.2.2. Row Count per Partition

In [12]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-2-row_count_per_partition.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-2-row_count_per_partition.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 tableoid :: regclass AS partition_name , 
 count ( * ) AS row_count 
 FROM 
 demo . amzn_reviews_partitioned 
 GROUP BY 1 
 ORDER BY 1 ;

21 rows affected.


partition_name,row_count
demo.amzn_reviews_partitioned_1_prt_year1995,199
demo.amzn_reviews_partitioned_1_prt_year1996,4677
demo.amzn_reviews_partitioned_1_prt_year1997,36722
demo.amzn_reviews_partitioned_1_prt_year1998,157533
demo.amzn_reviews_partitioned_1_prt_year1999,383823
demo.amzn_reviews_partitioned_1_prt_year2000,869772
demo.amzn_reviews_partitioned_1_prt_year2001,765379
demo.amzn_reviews_partitioned_1_prt_year2002,792835
demo.amzn_reviews_partitioned_1_prt_year2003,901016
demo.amzn_reviews_partitioned_1_prt_year2004,951572


#### 6.2.3. Row Count per Partition & Segment

In [13]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-3-row-count-per-partition-segment.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-3-row-count-per-partition-segment.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 tableoid :: regclass as partition_name , 
 gp_segment_id as segment_id , 
 count ( * ) AS segment_count , 
 SUM ( COUNT ( * )) OVER ( PARTITION BY tableoid :: regclass ) AS partition_count 
 FROM 
 demo . amzn_reviews_partitioned 
 GROUP BY 1 , 2 
 ORDER BY 1 , 2 ;

504 rows affected.


partition_name,segment_id,segment_count,partition_count
demo.amzn_reviews_partitioned_1_prt_year1995,0,10,199
demo.amzn_reviews_partitioned_1_prt_year1995,1,9,199
demo.amzn_reviews_partitioned_1_prt_year1995,2,6,199
demo.amzn_reviews_partitioned_1_prt_year1995,3,7,199
demo.amzn_reviews_partitioned_1_prt_year1995,4,8,199
demo.amzn_reviews_partitioned_1_prt_year1995,5,10,199
demo.amzn_reviews_partitioned_1_prt_year1995,6,7,199
demo.amzn_reviews_partitioned_1_prt_year1995,7,8,199
demo.amzn_reviews_partitioned_1_prt_year1995,8,11,199
demo.amzn_reviews_partitioned_1_prt_year1995,9,9,199


### 6.3. Partitioned Table Size and Disk Space Usage

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Compare the output below with the [Non-Partitioned Table Size and Disk Usage](http://127.0.0.1:9900/notebooks/gp-demo/AWS-GP-demo-2.ipynb#4.5.2.-Using-the-gp_toolkit-Administrative-Schema-(Greenplum-5.x)).

When you use `COPY` or `INSERT` to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [14]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-3-partitioned-table-size-disk.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-3-partitioned-table-size-disk.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT sotd . schemaname || '.' || sotd . tablename as tablename , 
 tabledisksize , 
 uncompressedsize , 
 tablesize , 
 indexsize , 
 toastsize , 
 othersize , 
 sopaid . partitionschemaname || '.' || sopaid . partitiontablename as partitionname , 
 sopaid . partitiontablesize , 
 sopaid . partitionindexsize 
 FROM ( 
 SELECT sotd . sotdoid as oid , 
 sotd . sotdschemaname as schemaname , 
 sotd . sotdtablename as tablename , 
 pg_size_pretty ( sotd . sotdsize :: BIGINT ) as tablesize , 
 pg_size_pretty ( sotd . sotdtoastsize :: BIGINT ) as toastsize , 
 pg_size_pretty ( sotd . sotdadditionalsize :: BIGINT ) as othersize 
 FROM 
 gp_toolkit . gp_size_of_table_disk as sotd 
 WHERE 
 sotd . sotdschemaname || '.' || sotd . sotdtablename = 'demo.amzn_reviews_partitioned' ) sotd 
 INNER JOIN ( 
 SELECT sotaid . sotaidoid as oid , 
 sotaid . sotaidschemaname as schemaname , 
 sotaid . sotaidtablename as tablename , 
 pg_size_pretty ( sotaid . sotaidtablesize :: BIGINT ) as tabledisksize , 
 pg_size_pretty ( sotaid . sotaididxsize :: BIGINT ) as indexsize 
 FROM 
 gp_toolkit . gp_size_of_table_and_indexes_disk as sotaid 
 WHERE 
 sotaid . sotaidschemaname || '.' || sotaid . sotaidtablename = 'demo.amzn_reviews_partitioned' ) sotaid 
 ON sotd . oid = sotaid . oid 
 INNER JOIN ( 
 SELECT sotu . sotuoid as oid , 
 sotu . sotuschemaname as schemaname , 
 sotu . sotutablename as tablename , 
 pg_size_pretty ( sotu . sotusize :: BIGINT ) as uncompressedsize 
 FROM 
 gp_toolkit . gp_size_of_table_uncompressed as sotu 
 WHERE 
 sotu . sotuschemaname || '.' || sotu . sotutablename = 'demo.amzn_reviews_partitioned' ) sotu 
 ON sotaid . oid = sotu . oid 
 RIGHT OUTER JOIN ( 
 SELECT 
 sopaidparentoid as parenttableoid , 
 pg_size_pretty ( sopaidpartitiontablesize :: BIGINT ) as partitiontablesize , 
 pg_size_pretty ( sopaidpartitionindexessize :: BIGINT ) as partitionindexsize , 
 sopaidpartitionschemaname as partitionschemaname , 
 sopaidpartitiontablename as partitiontablename 
 FROM 
 gp_toolkit . gp_size_of_partition_and_indexes_disk 
 WHERE 
 sopaidparentschemaname || '.' || sopaidparenttablename = 'demo.amzn_reviews_partitioned' ) sopaid 
 ON sopaid . parenttableoid = sotd . oid ;

21 rows affected.


tablename,tabledisksize,uncompressedsize,tablesize,indexsize,toastsize,othersize,partitionname,partitiontablesize,partitionindexsize
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1996,5152 kB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1995,1568 kB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1997,32 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1998,118 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1999,276 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2000,774 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2001,776 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2002,829 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2003,967 MB,0 bytes
demo.amzn_reviews_partitioned,800 kB,800 kB,0 bytes,0 bytes,800 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2004,1094 MB,0 bytes


### 6.4. Verify your Partition Strategy and Demonstrate *Partition Elimination* functionality

When a table is partitioned based on the query predicate, you can use `EXPLAIN` to verify that the query optimizer scans only the relevant data to examine the query plan. For example, the `demo.amzn_reviews_2` table is date-range partitioned by year. 

### Example 2: `SELECT` data for a single day (`2011-07-12`):

The query plan for this query should show a table scan of only the following tables:

- the default partition returning 0-1 rows (if your partition design has one)
- the 2011 partition (`	demo.amzn_reviews_2_1_prt_year2011`) returning ***some number*** of rows

To confirm, execute the `EXPLAIN` query and check the query plan:

In [15]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN SELECT * FROM demo . amzn_reviews_partitioned WHERE review_date = '2011-07-12' ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------- 
 Gather Motion 24 : 1 ( slice1 ; segments : 24 ) ( cost = 0.00..445.88 rows = 12768 width = 467 ) 
 -> Sequence ( cost = 0.00..431.62 rows = 532 width = 467 ) 
 -> Partition Selector for amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 10.00..100.00 rows = 5 width = 4 ) 
 Partitions selected : 1 ( out of 21 ) 
 -> Dynamic Table Scan on amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 0.00..431.62 rows = 532 width = 467 ) 
 Filter : review_date = '2011-07-12' :: date 
 Settings : optimizer_enable_associativity = on 
 Optimizer status : PQO version 3.52.0 
 ( 8 rows )

### Example 3 : "Single-Partition" `SELECT`

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Original/Non-Partitioned Table (`demo.amzn_reviews`)

In [16]:
query = !cat script/6-4-calendar-foundation.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
7374 rows affected.
Done.
Done.
7374 rows affected.
Done.
Done.


[]

In [17]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-3-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 cdate_mtd 
 FROM demo . amzn_reviews , 
 demo . calendar_mtd 
 WHERE 
 amzn_reviews . review_date = calendar_mtd . cdate_mtd 
 AND cdate = '2015-03-25' :: DATE 
 GROUP BY cdate_mtd 
 ORDER BY cdate_mtd ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 
 Gather Motion 24 : 1 ( slice4 ; segments : 24 ) ( cost = 0.00..3299.07 rows = 31 width = 12 ) 
 Merge Key : demo . calendar . cdate 
 Rows out : 25 rows at destination with 1691 ms to end . 
 -> GroupAggregate ( cost = 0.00..3299.07 rows = 2 width = 12 ) 
 Group By : demo . calendar . cdate 
 Rows out : Avg 1.4 rows x 18 workers . Max 2 rows ( seg1 ) with 1691 ms to end . 
 -> Sort ( cost = 0.00..3299.07 rows = 2 width = 12 ) 
 Sort Key : demo . calendar . cdate 
 Sort Method : quicksort Max Memory : 33 KB Avg Memory : 33 KB ( 24 segments ) 
 Rows out : Avg 33.3 rows x 18 workers . Max 48 rows ( seg1 ) with 1691 ms to end . 
 Executor memory : 58 K bytes avg , 58 K bytes max ( seg0 ) . 
 Work_mem used : 58 K bytes avg , 58 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 -> Redistribute Motion 24 : 24 ( slice3 ; segments : 24 ) ( cost = 0.00..3299.07 rows = 2 width = 12 ) 
 Hash Key : demo . calendar . cdate 
 Rows out : Avg 33.3 rows x 18 workers at destination . Max 48 rows ( seg1 ) with 1527 ms to first row , 1691 ms to end . 
 -> Result ( cost = 0.00..3299.07 rows = 2 width = 12 ) 
 Rows out : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 1561 ms to end . 
 -> HashAggregate ( cost = 0.00..3299.07 rows = 2 width = 12 ) 
 Group By : demo . calendar . cdate 
 Rows out : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 1561 ms to end . 
 Executor memory : 89 K bytes avg , 89 K bytes max ( seg0 ) . 
 ( seg0 ) Hash chain length 1.3 avg , 3 max , using 19 of 32 buckets ; total 0 expansions . 
 -> Hash Join ( cost = 0.00..3296.82 rows = 18061 width = 4 ) 
 Hash Cond : amzn_reviews . review_date = demo . calendar . cdate 
 Rows out : Avg 126774.9 rows x 24 workers . Max 127301 rows ( seg15 ) with 13 ms to first row , 1511 ms to end . 
 Executor memory : 1 K bytes avg , 1 K bytes max ( seg0 ) . 
 Work_mem used : 1 K bytes avg , 1 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 ( seg15 ) Hash chain length 1.0 avg , 1 max , using 25 of 524288 buckets . 
 -> Table Scan on amzn_reviews ( cost = 0.00..1628.26 rows = 4388766 width = 4 ) 
 Rows out : Avg 4297719.7 rows x 24 workers . Max 4301096 rows ( seg6 ) with 0.164 ms to first row , 1209 ms to end . 
 -> Hash ( cost = 862.20..862.20 rows = 31 width = 4 ) 
 Rows in : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 5.537 ms to end , start offset by 12 ms . 
 -> Broadcast Motion 24 : 24 ( slice2 ; segments : 24 ) ( cost = 0.00..862.20 rows = 31 width = 4 ) 
 Rows out : Avg 25.0 rows x 24 workers at destination . Max 25 rows ( seg0 ) with 1.493 ms to first row , 5.503 ms to end . 
 -> Hash Join ( cost = 0.00..862.20 rows = 2 width = 4 ) 
 Hash Cond : demo . calendar . from_mtd = demo . calendar . from_mtd AND demo . calendar . from_ytd = demo . calendar . from_ytd 
 Join Filter : demo . calendar . cdate >= demo . calendar . cdate 
 Rows out : Avg 1.4 rows x 18 workers . Max 2 rows ( seg1 ) with 7.381 ms to first row , 9.326 ms to end . 
 Executor memory : 1 K bytes avg , 1 K bytes max ( seg0 ) . 
 Work_mem used : 1 K bytes avg , 1 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 ( seg1 ) Hash chain length 1.0 avg , 1 max , using 1 of 1048576 buckets . 
 -> Table Scan on calendar ( cost = 0.00..431.01 rows = 308 width = 12 ) 
 Rows out : Avg 307.2 rows x 24 workers . Max 318 rows ( seg11 ) with 0.050 ms to first row , 0.067 ms to end . 
 -> Hash ( cost = 431.02..431.02 rows = 1 width = 12 ) 
 Rows in : Avg 1.0 rows x 24 workers . Max 1 rows ( seg0 ) with 0.040 ms to end , start offset by 12 ms . 
 -> Broadcast Motion 24 : 24 (

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Partitioned Table (`demo.amzn_reviews_partitioned`)

In [18]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-2.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output= !cat script/6-4-explain-example-3-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 cdate_mtd 
 FROM demo . amzn_reviews_partitioned , 
 demo . calendar_mtd 
 WHERE 
 amzn_reviews_partitioned . review_date = calendar_mtd . cdate_mtd 
 AND cdate = '2015-03-25' :: DATE 
 GROUP BY cdate_mtd 
 ORDER BY cdate_mtd ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 
 Gather Motion 24 : 1 ( slice4 ; segments : 24 ) ( cost = 0.00..3276.13 rows = 31 width = 12 ) 
 Merge Key : demo . calendar . cdate 
 Rows out : 25 rows at destination with 700 ms to end . 
 -> GroupAggregate ( cost = 0.00..3276.13 rows = 2 width = 12 ) 
 Group By : demo . calendar . cdate 
 Rows out : Avg 1.4 rows x 18 workers . Max 2 rows ( seg1 ) with 699 ms to end . 
 -> Sort ( cost = 0.00..3276.13 rows = 2 width = 12 ) 
 Sort Key : demo . calendar . cdate 
 Sort Method : quicksort Max Memory : 33 KB Avg Memory : 33 KB ( 24 segments ) 
 Rows out : Avg 33.3 rows x 18 workers . Max 48 rows ( seg1 ) with 699 ms to end . 
 Executor memory : 33 K bytes avg , 33 K bytes max ( seg0 ) . 
 Work_mem used : 33 K bytes avg , 33 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 -> Redistribute Motion 24 : 24 ( slice3 ; segments : 24 ) ( cost = 0.00..3276.13 rows = 2 width = 12 ) 
 Hash Key : demo . calendar . cdate 
 Rows out : Avg 33.3 rows x 18 workers at destination . Max 48 rows ( seg1 ) with 394 ms to first row , 699 ms to end . 
 -> Result ( cost = 0.00..3276.13 rows = 2 width = 12 ) 
 Rows out : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 401 ms to end . 
 -> HashAggregate ( cost = 0.00..3276.13 rows = 2 width = 12 ) 
 Group By : demo . calendar . cdate 
 Rows out : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 401 ms to end . 
 Executor memory : 89 K bytes avg , 89 K bytes max ( seg0 ) . 
 ( seg0 ) Hash chain length 1.3 avg , 3 max , using 19 of 32 buckets ; total 0 expansions . 
 -> Hash Join ( cost = 0.00..3273.91 rows = 17833 width = 4 ) 
 Hash Cond : amzn_reviews_partitioned . review_date = demo . calendar . cdate 
 Rows out : Avg 126774.9 rows x 24 workers . Max 127301 rows ( seg15 ) with 15 ms to first row , 394 ms to end . 
 Executor memory : 1 K bytes avg , 1 K bytes max ( seg0 ) . 
 Work_mem used : 1 K bytes avg , 1 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 ( seg15 ) Hash chain length 1.0 avg , 1 max , using 25 of 524288 buckets . 
 -> Dynamic Table Scan on amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 0.00..1615.52 rows = 4333351 width = 4 ) 
 Rows out : Avg 1132757.9 rows x 24 workers . Max 1134463 rows ( seg22 ) with 0.138 ms to first row , 285 ms to end . 
 Partitions scanned : Avg 1.0 ( out of 21 ) x 24 workers . Max 1 parts ( seg0 ) . 
 -> Hash ( cost = 100.00..100.00 rows = 5 width = 4 ) 
 Rows in : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 10 ms to end , start offset by 1.654 ms . 
 -> Partition Selector for amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 10.00..100.00 rows = 5 width = 4 ) 
 Rows out : Avg 25.0 rows x 24 workers . Max 25 rows ( seg0 ) with 5.243 ms to first row , 10 ms to end . 
 -> Broadcast Motion 24 : 24 ( slice2 ; segments : 24 ) ( cost = 0.00..862.20 rows = 31 width = 4 ) 
 Rows out : Avg 25.0 rows x 24 workers at destination . Max 25 rows ( seg0 ) with 4.633 ms to first row , 9.420 ms to end . 
 -> Hash Join ( cost = 0.00..862.20 rows = 2 width = 4 ) 
 Hash Cond : demo . calendar . from_mtd = demo . calendar . from_mtd AND demo . calendar . from_ytd = demo . calendar . from_ytd 
 Join Filter : demo . calendar . cdate >= demo . calendar . cdate 
 Rows out : Avg 1.4 rows x 18 workers . Max 2 rows ( seg1 ) with 11 ms to first row , 13 ms to end . 
 Executor memory : 1 K bytes avg , 1 K bytes max ( seg0 ) . 
 Work_mem used : 1 K bytes avg , 1 K bytes max ( seg0 ) . Workfile : ( 0 spilling ) 
 ( seg1 ) Hash chain length 1.0 avg , 1 max , using 1 of 1048576 buckets . 
 -> Table Scan on cale

## Continue to Part 4 of Greenplum Database  Concepts Explained, **[Table Storage Models](AWS-GP-demo-4.ipynb)**.